<h4>1. Применить к текстам лемматизацию, удаление стоп слов и токенизацию по словам</h4>


In [179]:
import re
import pymorphy3
from nltk.corpus import stopwords
from collections import Counter
import numpy as np
from math import log


In [180]:
texts = [
    "Старик сидел у моря и смотрел на горизонт.",
    "Море тихо шептало, накатываясь на берег.",
    "Берег был пуст, кроме одинокого силуэта на песке."
]

In [181]:
morph = pymorphy3.MorphAnalyzer()
stop_words = set(stopwords.words('russian'))

In [182]:
def preprocessing(text):
    text = text.lower()
    text = re.sub(r'[^а-яё\s]', '', text)
    
    words = text.split()
    lemmas = [morph.parse(word)[0].normal_form for word in words 
             if word not in stop_words and word.strip()]
    
    return lemmas

In [183]:
for i in texts:
    print(preprocessing(i))


['старик', 'сидеть', 'море', 'смотреть', 'горизонт']
['море', 'тихо', 'шептать', 'накатываться', 'берег']
['берег', 'пустой', 'кроме', 'одинокий', 'силуэт', 'песок']


<h4>2. Реализовать Bag of Words</h4>


In [184]:
def create_vocabulary(texts):
    vocabulary = set()
    
    for text in texts:
        processed_text = preprocessing(text)
        vocabulary.update(processed_text)

    return sorted(list(vocabulary))

In [185]:
def create_bow_matrix(texts):
    vocabulary = create_vocabulary(texts)
    vocab_to_idx = {word: idx for idx, word in enumerate(vocabulary)}
    bow_matrix = []
    
    for text in texts:
        processed_text = preprocessing(text)
        vector = [0] * len(vocabulary)
        word_counts = Counter(processed_text)
        for word, count in word_counts.items():
            if word in vocab_to_idx:
                vector[vocab_to_idx[word]] = count
        
        bow_matrix.append(vector)
    
    return bow_matrix, vocabulary

In [186]:
bow_matrix, vocabulary = create_bow_matrix(texts)
bow_matrix = np.array(bow_matrix)

In [187]:
for word in vocabulary:
    print(word, end=", ")
    
print("\n")
print(bow_matrix)

берег, горизонт, кроме, море, накатываться, одинокий, песок, пустой, сидеть, силуэт, смотреть, старик, тихо, шептать, 

[[0 1 0 1 0 0 0 0 1 0 1 1 0 0]
 [1 0 0 1 1 0 0 0 0 0 0 0 1 1]
 [1 0 1 0 0 1 1 1 0 1 0 0 0 0]]


<h4>3. Реализовать TF-IDF</h4>


In [188]:
def compute_tfidf(bow_matrix):
    tf = bow_matrix / bow_matrix.sum(axis=1, keepdims=True)
    
    n_docs = len(bow_matrix)
    word_doc_counts = np.count_nonzero(bow_matrix, axis=0)
    idf = np.log(n_docs / (1 + word_doc_counts))

    tfidf = tf * idf
    return tfidf

In [189]:
tfidf_matrix = compute_tfidf(bow_matrix)

In [191]:
print(tfidf_matrix)

[[0.         0.08109302 0.         0.         0.         0.
  0.         0.         0.08109302 0.         0.08109302 0.08109302
  0.         0.        ]
 [0.         0.         0.         0.         0.08109302 0.
  0.         0.         0.         0.         0.         0.
  0.08109302 0.08109302]
 [0.         0.         0.06757752 0.         0.         0.06757752
  0.06757752 0.06757752 0.         0.06757752 0.         0.
  0.         0.        ]]


In [192]:
for i, vector in enumerate(tfidf_matrix):
    for word, score in zip(vocabulary, vector):
        if score > 0:
            print(f"  {word}: {score:.4f}")
    print()

  горизонт: 0.0811
  сидеть: 0.0811
  смотреть: 0.0811
  старик: 0.0811

  накатываться: 0.0811
  тихо: 0.0811
  шептать: 0.0811

  кроме: 0.0676
  одинокий: 0.0676
  песок: 0.0676
  пустой: 0.0676
  силуэт: 0.0676

